# ТМ на корпусах разных размеров наивной поэзии


Для корпуса были отобраны только стихи длиной 4 и более строк с одним метром для всех строк. Файлы не разделялись по типам окончаний (м/ж/д). Один документ представлен одной строфой стихотворения.

In [1]:
!pip3 install pymorphy2

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 8.2MB 6.7MB/s 


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import files

import re
import os
import pandas as pd
import csv

from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Определение стоп-слов: к списку, по умолчанию установленному для русского языка библиотекой NLTK, были добавлены некоторые, специфичные для нашего корпуса. 

In [ ]:
stopwords_ = stopwords.words('russian')
stopwords_.extend(['я', 'ты', 'он', 'она', 'твой', 'мой', 'они', 'это', 'наш', 'ваш',
                   'весь', 'все', 'свой', 'всё', 'ещё', 'оно', 'её', 'амж', 'амд',
                   'амма','анм', 'анж', 'анды', 'хм', 'хж', 'хд', 'яж', 'ям', 'яга', 'яд',
                   'дм', 'дж', 'дд', 'рецензия', 'ру', 'нибыть', 'лишь', 'очень', 'пусть'])

Загружаем файлы с корпусами

In [ ]:
os.mkdir('/content/corpus')
os.chdir('/content/corpus')

In [ ]:
uploaded = files.upload()

Saving Х4_жмжм.txt to Х4_жмжм.txt
Saving Х3_жмжм.txt to Х3_жмжм.txt
Saving Я4_дмдм.txt to Я4_дмдм.txt
Saving Х4_дмдм.txt to Х4_дмдм.txt
Saving Д4_дддд.txt to Д4_дддд.txt


In [ ]:
files = os.listdir()
files

['Х4_дмдм.txt', 'Д4_дддд.txt', 'Х3_жмжм.txt', 'Я4_дмдм.txt', 'Х4_жмжм.txt']

Оформляем информацию о корпусах в датафрейм

In [ ]:
meter_names = []
for i in files:
  name = re.search('(.+_\w{4})', i)
  meter_names.append(name.group(1))
res_df = pd.DataFrame({'meter': meter_names})
res_df

,meter
0,Х4_дмдм
1,Д4_дддд
2,Х3_жмжм
3,Я4_дмдм
4,Х4_жмжм


Определяем функции для обработки текста и модели

In [ ]:
def preprocess_text(texts_list):
  texts_list = [t for t in texts_list if t]   # getting rid of empty instances
  texts = []
  for tl in texts_list:
    check_meter = re.findall('\t.+\n', tl)
    check_lines = re.findall('.+\n', tl)
    if len(check_meter) == len(check_lines) and len(check_lines) > 3:
      texts.append(tl)
  texts = [re.sub('\d+', '', each) for each in texts] # getting rid of numbers
  texts = [re.sub('[A-Za-z]', '', each) for each in texts]   # getting rid of links
  texts = [re.sub('`', '', each) for each in texts] # getting rid of accentuation
  texts = [re.sub('\t.+\n', ' ', each) for each in texts] # getting rid of meters
  texts = [t for t in texts if t]   # getting rid of empty instances

  new_texts = []
  for t in range(len(texts)):
    new_words = []
    new_text = re.findall('\w+', texts[t])
    for w in range(len(new_text)):
      new_words.append(morph.parse(new_text[w])[0].normal_form)
    n_text = " ".join(new_words)
    new_texts.append(n_text)
  return new_texts

In [ ]:
def model(val):   # defining the function that will be applied to the dataframe to get results for each meter
  filename = val + '.txt'  # generating filename for each meter
  with open(filename, 'r', encoding='utf-8') as fi:
    meter_corpus = fi.read()  # getting the corpus for a specific meter

  texts = []
  meter_texts = meter_corpus.split('=====')   # getting an array of documents
  for i in meter_texts:
    st = i.split('\n\n')  # splitting each poem by paragraph, not mandatory
    for s in st:
      texts.append(s)

  preprocessed_texts = preprocess_text(texts)  # lemmatization etc.

  # setting up the model:
  tfidf_vectorizer = TfidfVectorizer(max_df=0.6, min_df=0.01, stop_words=stopwords_)
  tfidf = tfidf_vectorizer.fit_transform(preprocessed_texts)
  tfidf_feature_names = tfidf_vectorizer.get_feature_names()
  nmf = NMF(n_components=10).fit(tfidf)
  out = ''
  for idx, topic in enumerate(nmf.components_):
    out += (" ".join(tfidf_feature_names[i] for i in topic.argsort()[:-10 - 1:-1]))
    out += '\n'
  return out

Добавляем результаты в датафрейм

In [ ]:
res_df['topics'] = res_df['meter'].progress_apply(model)
res_df


100%|██████████| 5/5 [05:06<00:00, 61.29s/it]


,meter,topics
0,Х4_дмдм,любовь вновь прекрасный словно краска стих взг...
1,Д4_дддд,солнце ветер небо свет туча земля утро трава л...
2,Х3_жмжм,глаз рука слеза милый стать слово губа взгляд ...
3,Я4_дмдм,небо весна рука свет ночь ветер солнце прийти ...
4,Х4_жмжм,стать хотеть жить мир человек бог сказать дать...


Записываем результаты в текстовый и csv-файл

In [ ]:
with open('results.txt', 'a', encoding='utf-8') as file:
  for i in range(len(res_df['meter'])):
    file.write(res_df['meter'][i] + '\n\n' + res_df['topics'][i] + '=====\n')

In [ ]:
with open('results.txt', 'r', encoding='utf-8') as file:
    text = file.read()

meters = text.split('=====')
dict_ = {}
meters = [met for met in meters if met]
del meters[-1]
for m in meters:
    meter = re.findall('(.+)\n\n', m)[0]
    topics = re.findall('\n\n([\s|\S]+)', m)[0]
    topic_list = topics.split('\n')
    topic_list = [t for t in topic_list if t]
    tuple_topics = tuple(topic_list)
    dict_.update({meter: tuple_topics})

topic_numbers = []
for l in range(1, 11):
    topic_numbers.append('topic ' + str(l))
for i in dict_.keys():
    with open('ямб10_strict.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow([i])
        writer.writerow(topic_numbers)
        topics = dict_[i]
        new_list = []
        for k in topics:
            words = k.split(' ')
            new_list.append(words)
        out = zip(*new_list)
        for o in out:
            writer.writerow(o)
        writer.writerow([''])